In [173]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as dt
import pandas as pd 
import numpy as np
import torch.utils.data as data_utils

# Input and Hyperparameters #

In [174]:
data_path = "./US_length.csv"
learning_rate = 0.01
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
split = 0.8
batch_size = 50
num_epochs = 50
subset_val = 1000

# Data Cleaning #

In [175]:
df_main = pd.read_csv(data_path, sep = ',')
#subset data for testing ,use the first 10000
df = df_main[:subset_val]
secs = df['sec'].values
mins = df['min'].values
hours = df['hour'].values
total_time = [s+60*m+3600*h for s,m,h in zip(secs, mins, hours)]
df['total_time'] = total_time
df_features=df[['title', 'total_time']]

C:\Users\ChuckDaddy\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [176]:
df_features['title_upper_count'] = df_features['title'].str.findall(r'[A-Z]').str.len()
df_features['tag_count'] = [len(s.split('|')) for s in df['tags'].values]
df_features['category_id'] = df_main['category_id']
df_label = pd.DataFrame()
df_label['views'] = df['views']
df_features = df_features.drop(columns = 'title')

C:\Users\ChuckDaddy\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\ChuckDaddy\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\ChuckDaddy\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [177]:
df_features.head()


,total_time,title_upper_count,tag_count,category_id
0,678,28,1,22
1,1431,11,4,24
2,569,8,23,23
3,430,4,27,24
4,733,12,14,24


In [178]:
df_label.head()


,views
0,748374
1,2418783
2,3191434
3,343168
4,2095731


# Setup Dataloader #

In [179]:
tv_split = int(len(df_features)*split)
train_set = df_features[:tv_split]
train_label = df_label[:tv_split]
validation_set = df_features[tv_split:]
validation_label = df_label[tv_split:]


In [180]:
class Dataset(dt.Dataset):
    def __init__(self, feature, labels):
        self.labels = labels
        self.feature = feature
    def __len__(self):
        return(len(self.feature))
    def __getitem__(self, idx):
        cur_feature = self.feature.iloc[idx]
        cur_feature = np.array(cur_feature)
        cur_label = self.labels.iloc[idx]
        cur_label = np.array(cur_label)
        sample = (cur_feature, cur_label)
        return sample

In [181]:
train = Dataset(train_set, train_label)
train_loader = data_utils.DataLoader(train, batch_size, shuffle = True)
test = Dataset(validation_set, validation_label)
test_loader = data_utils.DataLoader(test, batch_size, shuffle = True)

# Define Neural Network

In [182]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size).double() 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes).double()
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        return out

# Main #

In [183]:
input_size = len(df_features.iloc[0])
output_size = len(df_label.columns)
hidden_size = int(np.ceil((input_size+output_size)/2))
model = NeuralNet(input_size, hidden_size, output_size).to(device)
# Loss and optimizer
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  
# Train the model
total_step = len(train_loader)
print(total_step)
loss = torch.tensor(0.0)
for epoch in range(num_epochs):
    curr_epoch_loss = 0.0
    for i, (feature, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        # Move tensors to the configured device
        feature = torch.tensor(feature,  dtype = torch.float64).to(device)
        label = torch.tensor(labels,  dtype = torch.float64).to(device)

#         # Forward pass/
        outputs = model(feature)
#         print(outputs, label)
        loss = criterion(outputs, label)
        curr_epoch_loss+=loss.data.item()
        loss.backward()
        optimizer.step()
#         print(loss.data.item())
    print(curr_epoch_loss)
#         
        
#         # Backward and optimize
#         
#         loss.backward()
#         optimizer.step()
        
        #if (i+1) % 100 == 0:
#         print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.1f}' 
#                    .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model

16


C:\Users\ChuckDaddy\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\ChuckDaddy\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


16374040.945416743
16372082.654164676
16367807.466266887
16361382.450476317
16355895.479325688
16349737.368237665
16343159.682456296
16340501.576520996
16338623.27303585
16336313.60026027
16334309.711083576
16332700.24900774
16331625.450527806
16330400.712368224
16329239.254130252
16327827.722968882
16326534.55882909
16325436.873092573
16323856.021937827
16322995.942388559
16320973.67964404
16319210.24939458
16317505.023111816
16315868.697992576
16313906.698765492
16311951.740959056
16309894.43609395
16307600.6506391
16305353.192911949
16302814.472724041
16300318.346800977
16297653.392287744
16294917.657099202
16291672.479018468
16288473.963264413
16285036.773849055
16281668.160447516
16277690.844980711
16273878.491489166
16270066.505325789
16265572.34860107
16261520.845963886
16256413.990004595
16251554.809156131
16246648.086080447
16241510.526672125
16236902.388550943
16230098.87282066
16224837.202526247
16218383.117326703


In [162]:
# for i, (feature, labels) in enumerate(train_loader):  
#     if(i >10):
#         break
#     print((type(feature)))
#     print(labels)